# **Deep Learning For Lung Cancer Prediction Based on Transcriptomic Data : MLP with Embeddings learned by a VAE and Transfer Learning**

> Author : **Aymen MERROUCHE**. <br>
> In this notebook, we implement an **MLP with VAE computed embeddings** for our binary classification task. First we train, on the Non Lung Cancer Dataset, a variational autoencoder to learn embeddings for our transcriptomic data. Then we pre-train an MLP on Non Lung Data and finally we fine-tune the latter on the Lung Cancer Data.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import matplotlib.pyplot as plt
import numpy as np
from torch.distributions.normal import Normal
import warnings
warnings.filterwarnings('ignore')
from modules.MLP import *
# device, cuda if available else use cpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
from utils import *
from train import *
from data_utils import *
from modules.deformable_cnn import *
from modules.focal_loss import *
%load_ext autoreload
%autoreload 2

# **1 - Get The Data :**

In [3]:
# load hyperparametrs
# data paths args
with open('./configs/data_paths.yaml', 'r') as stream:
    data_paths_args  = yaml.load(stream,Loader=yaml.Loader)
    
    
# basic cnn args
with open('./configs/vae_mlp.yaml', 'r') as stream:
    vae_mlp_args  = yaml.load(stream,Loader=yaml.Loader)

In [4]:
%%time
# Getting the data
# dataset for non lung
non_lung_dataset = TranscriptomicVectorsDatasetNonLung(data_paths_args["path_to_pan_cancer_hdf5_files"])
non_lung_dataloader_train, non_lung_dataloader_validation = get_data_loaders(non_lung_dataset, batch_size_train = vae_mlp_args["batch_size_pt"],\
                                                                             batch_size_validation = vae_mlp_args["batch_size_pt"], test_proportion = 0.0)

CPU times: user 13 s, sys: 374 ms, total: 13.4 s
Wall time: 12.9 s


In [5]:
%%time
# Getting the data
# dataset for non lung
non_lung_dataloader_train_splitted, non_lung_dataloader_validation_splitted = get_data_loaders(non_lung_dataset, batch_size_train = vae_mlp_args["batch_size_pt"],\
                                                                             batch_size_validation = vae_mlp_args["batch_size_pt"])

CPU times: user 657 ms, sys: 74 µs, total: 657 ms
Wall time: 655 ms


In [6]:
%%time
# Getting the data
# dataset for lung
lung_dataset = TranscriptomicVectorsDatasetLung(data_paths_args["path_to_pan_cancer_hdf5_files"])
lung_dataloader_train, lung_dataloader_validation = get_data_loaders(lung_dataset, batch_size_train = vae_mlp_args["batch_size_ft"],\
                                                                             batch_size_validation = vae_mlp_args["batch_size_ft"])

CPU times: user 11.3 s, sys: 167 ms, total: 11.5 s
Wall time: 11.4 s


# **2 - Get The Embeddings with the VAE :**

## 1- Encoder and Decoder and VAE with linear layers :

In [7]:
class Encoder(nn.Module):
    """
    VAE Encoder with linear layers
    """
    def __init__(self, dim_input, hidden, dim_z):
        super().__init__()
        self.lin1 = nn.Linear(dim_input, hidden)
        # generate covariance matrix (hyp : diagonal)
        self.sigma = nn.Linear(hidden, dim_z)
        # genrate mean vector 
        self.mu = nn.Linear(hidden, dim_z)
    def forward(self, x):
        lin1 = F.relu(self.lin1(x))
        #we infer log(sigma**2) for stability reasons
        log_sigma_squared = self.sigma(lin1) 
        mu = self.mu(lin1)
        return mu, log_sigma_squared

In [8]:
class Decoder(nn.Module):
    """
    VAE Decoder with linear layers
    """
    def __init__(self, dim_input, hidden, dim_z):
        super().__init__()
        self.lin1 = nn.Linear(dim_z, hidden)
        self.lin2 = nn.Linear(hidden, dim_input)
    def forward(self, x):
        lin1 = F.relu(self.lin1(x))
        x = torch.sigmoid(self.lin2(lin1))
        return x  

In [9]:
class VAE(nn.Module):
    """
    VAE with linear layers based Encoder and Decoder
    """
    def __init__(self, dim_input, hidden, dim_z):
        super().__init__()
        # encoder
        self.encoder = Encoder(dim_input, hidden, dim_z).to(device)
        # decoder
        self.decoder = Decoder(dim_input, hidden, dim_z).to(device)
        
    def forward(self, x):
        mu, log_sigma_squared = self.encoder(x)
        z = self.get_codes(mu, log_sigma_squared)
        return self.decoder(z), mu, log_sigma_squared

    def get_codes(self, mu, log_sigma_squared):
        # input is log(sigma**2)
        # log(sigma**2) = 2 * log(sigma)
        # log(sigma) = (1/2)*log(sigma**2)
        # sigma = exp((1/2)*log(sigma**2))
        sigma = torch.exp((1/2) * log_sigma_squared)
        # reparametrisation trick
        eps = torch.empty_like(sigma).normal_().to(device)
        return eps.mul(sigma).add_(mu)
    def sample_data(self,z):
        return self.decoder(z)

In [10]:
def variational_lower_bound(oringinal, reconstructed, mu, log_sigma_squared):
    reconstructed = reconstructed
    oringinal = oringinal
    reconstruction_term = F.binary_cross_entropy(reconstructed, oringinal, reduction='sum')
    kl_div = - (1/2) * torch.sum(1 + log_sigma_squared - mu.pow(2) - log_sigma_squared.exp())
    return reconstruction_term + kl_div

In [19]:
# dimension of the latent variable z
dim_z = 300
# dimnesion of the generator's output/discriminator's input (an mnist image)
dim_input = len(non_lung_dataset[0][0])
# dimension of hidden layers
hidden = 256

In [20]:
vae = VAE(dim_input, hidden, dim_z).to(device).double()
print(vae)

VAE(
  (encoder): Encoder(
    (lin1): Linear(in_features=7509, out_features=256, bias=True)
    (sigma): Linear(in_features=256, out_features=300, bias=True)
    (mu): Linear(in_features=256, out_features=300, bias=True)
  )
  (decoder): Decoder(
    (lin1): Linear(in_features=300, out_features=256, bias=True)
    (lin2): Linear(in_features=256, out_features=7509, bias=True)
  )
)


## 2 - Training The VAE :

In [21]:
# parameters
epochs = 30

# optimisation
lr = 0.001

# optimizer
optim_vae = optim.Adam(vae.parameters())

# noise to display samples during training
noise = torch.randn(vae_mlp_args["batch_size_pt"], dim_z, device=device)

In [22]:
# per epoch logging containers
v_lower_bound = []
image_and_reconstruction = []
sampled_images = []
for epoch in range(epochs):
    print("EPOCH : ",epoch)
    # per gradient step logging containers
    epoch_v_lower_bound = []
    
    for i, (vectors, labels) in enumerate(non_lung_dataloader_train):
        
        # perform a gradient step
        
        # flatten and push to device
        vectors = vectors.to(device).double()
        
        # run vae
        reconstruction, mu, log_sigma_squared = vae(vectors)

        # loss
        loss = variational_lower_bound(vectors, reconstruction, mu, log_sigma_squared)
        
        # backpropagation
        optim_vae.zero_grad()
        loss.backward()
        optim_vae.step()
        
        # logging
        epoch_v_lower_bound.append(loss.item())
        
        # display stats and an image during training
        """if i % 300 == 0 :
            print("BATCH :",i)
            print("| v_lower_bound: ", epoch_v_lower_bound[i])"""

    # per epoch logging
    v_lower_bound.append(sum(epoch_v_lower_bound) / len(epoch_v_lower_bound))

EPOCH :  0
EPOCH :  1
EPOCH :  2
EPOCH :  3
EPOCH :  4
EPOCH :  5
EPOCH :  6
EPOCH :  7
EPOCH :  8
EPOCH :  9
EPOCH :  10
EPOCH :  11
EPOCH :  12
EPOCH :  13
EPOCH :  14
EPOCH :  15
EPOCH :  16
EPOCH :  17
EPOCH :  18
EPOCH :  19
EPOCH :  20
EPOCH :  21
EPOCH :  22
EPOCH :  23
EPOCH :  24
EPOCH :  25
EPOCH :  26
EPOCH :  27
EPOCH :  28
EPOCH :  29


In [23]:
# Fine tuning the VAE
# per epoch logging containers
v_lower_bound = []
image_and_reconstruction = []
sampled_images = []
for epoch in range(epochs):
    print("EPOCH : ",epoch)
    # per gradient step logging containers
    epoch_v_lower_bound = []
    
    for i, (vectors, labels) in enumerate(lung_dataloader_train):
        
        # perform a gradient step
        
        # flatten and push to device
        vectors = vectors.to(device).double()
        
        # run vae
        reconstruction, mu, log_sigma_squared = vae(vectors)

        # loss
        loss = variational_lower_bound(vectors, reconstruction, mu, log_sigma_squared)
        
        # backpropagation
        optim_vae.zero_grad()
        loss.backward()
        optim_vae.step()
        
        # logging
        epoch_v_lower_bound.append(loss.item())
        
        # display stats and an image during training
        """if i % 300 == 0 :
            print("BATCH :",i)
            print("| v_lower_bound: ", epoch_v_lower_bound[i])"""

    # per epoch logging
    v_lower_bound.append(sum(epoch_v_lower_bound) / len(epoch_v_lower_bound))

EPOCH :  0
EPOCH :  1
EPOCH :  2
EPOCH :  3
EPOCH :  4
EPOCH :  5
EPOCH :  6
EPOCH :  7
EPOCH :  8
EPOCH :  9
EPOCH :  10
EPOCH :  11
EPOCH :  12
EPOCH :  13
EPOCH :  14
EPOCH :  15
EPOCH :  16
EPOCH :  17
EPOCH :  18
EPOCH :  19
EPOCH :  20
EPOCH :  21
EPOCH :  22
EPOCH :  23
EPOCH :  24
EPOCH :  25
EPOCH :  26
EPOCH :  27
EPOCH :  28
EPOCH :  29


# **3 - Pre-Training on the Non Lung Cancer Dataset :**

In [24]:
# network
net = MLP(input_size = dim_z, layers = [160, 100]).to(device).double()

# loss and optimizer  
criterion = FocalLoss().to(device)
optimizer = optim.Adam(net.parameters(), lr=vae_mlp_args['lr_pt'])

# Logging + Experiment

ignore_keys = {'no_tensorboard'}
# get hyperparameters with values in a dict
hparams = {**vae_mlp_args}
# generate a name for the experiment
expe_name = '_'.join([f"{key}={val}" for key, val in hparams.items()])
print("Experimenting with : \n \t"+expe_name)
# path where to save the model
savepath = Path('/tempory/transcriptomic_data/pre_trained_mlp_vae_checkpt.pt')
# Tensorboard summary writer
if vae_mlp_args['no_tensorboard']:
    writer = None
else:
    writer = SummaryWriter("runs/runs"+"_"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+expe_name)
# start the experiment
checkpoint = CheckpointState(net, optimizer, savepath=savepath)
fit(checkpoint, criterion, lung_dataloader_train, lung_dataloader_validation, vae_mlp_args['epochs'], writer=writer, embedding_computer=vae)
if not vae_mlp_args['no_tensorboard']:
    writer.close()

Epoch 1/50: 100%|██████████| 5/5 [00:00<00:00, 55.72it/s, loss=1.9078e-01]

Experimenting with : 
 	epochs=50_batch_size_pt=128_lr_pt=0.001_batch_size_ft=128_lr_ft=0.001_no_tensorboard=True
Training on GPU 




Epoch 3/50:   0%|          | 0/5 [00:00<?, ?it/s, loss=1.2240e-01]

Epoch 1/50, Train Loss: 1.1266e-01, Test Loss: 0.0584
Epoch 1/50, Train Accuracy: 67.86%, Test Accuracy: 80.31%
Epoch 1/50, Train AUC: 72.55%, Test AUC: 65.28%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.94      0.84      0.89       260
      Cancer       0.15      0.32      0.20        22

    accuracy                           0.80       282
   macro avg       0.54      0.58      0.54       282
weighted avg       0.87      0.80      0.83       282

Epoch 2/50, Train Loss: 8.8825e-02, Test Loss: 0.0665
Epoch 2/50, Train Accuracy: 78.28%, Test Accuracy: 76.16%
Epoch 2/50, Train AUC: 79.21%, Test AUC: 65.38%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.94      0.79      0.86       260
      Cancer       0.13      0.36      0.19        22

    accuracy                           0.76       282
   macro avg       0.53      0.58      0.52       282
weighted 

Epoch 4/50: 100%|██████████| 5/5 [00:00<00:00, 71.53it/s, loss=8.6326e-02]


...

Epoch 46/50, Train Loss: 9.8649e-03, Test Loss: 0.0729
Epoch 46/50, Train Accuracy: 93.08%, Test Accuracy: 77.24%
Epoch 46/50, Train AUC: 97.25%, Test AUC: 59.13%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.93      0.85      0.89       260
      Cancer       0.11      0.23      0.15        22

    accuracy                           0.80       282
   macro avg       0.52      0.54      0.52       282
weighted avg       0.86      0.80      0.83       282

Epoch 47/50, Train Loss: 1.1845e-02, Test Loss: 0.0615
Epoch 47/50, Train Accuracy: 92.70%, Test Accuracy: 79.05%
Epoch 47/50, Train AUC: 97.33%, Test AUC: 57.12%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.92      0.88      0.90       260
      Cancer       0.03      0.05      0.04        22

    accuracy                           0.81       282
   macro avg       0.47      0.46      0.47       282
wei

Epoch 49/50: 100%|██████████| 5/5 [00:00<00:00, 63.52it/s, loss=1.3815e-02]


Epoch 48/50, Train Loss: 1.2882e-02, Test Loss: 0.0634
Epoch 48/50, Train Accuracy: 92.94%, Test Accuracy: 78.79%
Epoch 48/50, Train AUC: 97.49%, Test AUC: 53.37%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.92      0.87      0.89       260
      Cancer       0.06      0.09      0.07        22

    accuracy                           0.81       282
   macro avg       0.49      0.48      0.48       282
weighted avg       0.85      0.81      0.83       282

Epoch 49/50, Train Loss: 1.2103e-02, Test Loss: 0.0641
Epoch 49/50, Train Accuracy: 90.70%, Test Accuracy: 79.27%
Epoch 49/50, Train AUC: 97.19%, Test AUC: 52.97%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.92      0.85      0.88       260
      Cancer       0.05      0.09      0.06        22

    accuracy                           0.79       282
   macro avg       0.48      0.47      0.47       282
wei

Epoch 50/50: 100%|██████████| 5/5 [00:00<00:00, 62.95it/s, loss=1.9078e-02]

Epoch 50/50, Train Loss: 1.0654e-02, Test Loss: 0.0717
Epoch 50/50, Train Accuracy: 93.33%, Test Accuracy: 80.07%
Epoch 50/50, Train AUC: 97.69%, Test AUC: 55.98%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.92      0.87      0.90       260
      Cancer       0.08      0.14      0.10        22

    accuracy                           0.81       282
   macro avg       0.50      0.50      0.50       282
weighted avg       0.86      0.81      0.83       282


Finished.
Best validation AUC: 6.8619e-01
With accuracy: 0.756209929784139


# **4 - Fine-Tuning on the Lung Cancer Dataset :**

In [25]:
# Load the pretrained Model
net =  MLP(input_size = dim_z, layers = [160, 100]).to(device).double()
optimizer = optim.Adam(net.parameters(), lr=vae_mlp_args['lr_ft'])
# path where the pre-trained model is saved : defined above+"_best"
savepath = Path('/tempory/transcriptomic_data/pre_trained_mlp_vae_checkpt_best.pt')
checkpoint = CheckpointState(net, optimizer, savepath=savepath)
checkpoint.load()
pretrained = checkpoint.model

In [26]:
# Beginnig Of Transfer Learnig Procedure
net = fine_tune_mlp(pretrained)
net = net.to(device).double()
criterion = FocalLoss().to(device)
optimizer = optim.Adam(net.parameters(), lr=vae_mlp_args['lr_ft'])
savepath = Path('/tempory/transcriptomic_data/fine_tuned_mlp_vae_checkpt.pt')
checkpoint = CheckpointState(net, optimizer, savepath=savepath)
fit(checkpoint, criterion, lung_dataloader_train, lung_dataloader_validation, vae_mlp_args['epochs'], embedding_computer=vae)

Epoch 1/50: 100%|██████████| 5/5 [00:00<00:00, 63.45it/s, loss=1.1517e-01]

Training on GPU 




Epoch 2/50:   0%|          | 0/5 [00:00<?, ?it/s, loss=1.2011e-01]

Epoch 1/50, Train Loss: 7.6504e-02, Test Loss: 0.0730
Epoch 1/50, Train Accuracy: 78.49%, Test Accuracy: 76.66%
Epoch 1/50, Train AUC: 82.78%, Test AUC: 65.07%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.94      0.78      0.85       260
      Cancer       0.12      0.36      0.19        22

    accuracy                           0.75       282
   macro avg       0.53      0.57      0.52       282
weighted avg       0.87      0.75      0.80       282



Epoch 3/50:   0%|          | 0/5 [00:00<?, ?it/s, loss=9.5453e-02]

...

Epoch 45/50, Train Loss: 7.6739e-03, Test Loss: 0.0564
Epoch 45/50, Train Accuracy: 91.94%, Test Accuracy: 81.09%
Epoch 45/50, Train AUC: 97.89%, Test AUC: 58.81%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.92      0.87      0.90       260
      Cancer       0.06      0.09      0.07        22

    accuracy                           0.81       282
   macro avg       0.49      0.48      0.48       282
weighted avg       0.85      0.81      0.83       282

Epoch 46/50, Train Loss: 4.3887e-03, Test Loss: 0.0633
Epoch 46/50, Train Accuracy: 97.17%, Test Accuracy: 79.31%
Epoch 46/50, Train AUC: 98.51%, Test AUC: 56.43%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.92      0.88      0.90       260
      Cancer       0.06      0.09      0.07        22

    accuracy                           0.82       282
   macro avg       0.49      0.48      0.48       282
wei

Epoch 49/50:   0%|          | 0/5 [00:00<?, ?it/s]

Epoch 47/50, Train Loss: 5.6826e-03, Test Loss: 0.0579
Epoch 47/50, Train Accuracy: 94.34%, Test Accuracy: 79.29%
Epoch 47/50, Train AUC: 97.89%, Test AUC: 56.84%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.92      0.86      0.89       260
      Cancer       0.07      0.14      0.10        22

    accuracy                           0.80       282
   macro avg       0.50      0.50      0.49       282
weighted avg       0.86      0.80      0.83       282

Epoch 48/50, Train Loss: 6.1912e-03, Test Loss: 0.0789
Epoch 48/50, Train Accuracy: 95.75%, Test Accuracy: 77.50%
Epoch 48/50, Train AUC: 98.10%, Test AUC: 53.23%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.91      0.87      0.89       260
      Cancer       0.03      0.05      0.04        22

    accuracy                           0.80       282
   macro avg       0.47      0.46      0.46       282
wei

Epoch 50/50: 100%|██████████| 5/5 [00:00<00:00, 57.33it/s, loss=9.8304e-03]


Epoch 49/50, Train Loss: 5.9555e-03, Test Loss: 0.0743
Epoch 49/50, Train Accuracy: 95.77%, Test Accuracy: 81.35%
Epoch 49/50, Train AUC: 98.43%, Test AUC: 53.13%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.92      0.88      0.90       260
      Cancer       0.06      0.09      0.07        22

    accuracy                           0.82       282
   macro avg       0.49      0.48      0.48       282
weighted avg       0.85      0.82      0.83       282

Epoch 50/50, Train Loss: 5.8518e-03, Test Loss: 0.0837
Epoch 50/50, Train Accuracy: 95.44%, Test Accuracy: 78.79%
Epoch 50/50, Train AUC: 98.23%, Test AUC: 51.71%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.92      0.87      0.89       260
      Cancer       0.06      0.09      0.07        22

    accuracy                           0.81       282
   macro avg       0.49      0.48      0.48       282
wei